# A Demonstration of Exploitation/Exploration Trade-off
## Inspired by the following notebook:
[this notebook](https://github.com/fmfn/BayesianOptimization/blob/master/examples/exploitation%20vs%20exploration.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

In [ ]:
# local modules
import sys; sys.path.append('..')
import optimisation as op

In [ ]:
# Make deterministic
np.random.seed(42)

In [ ]:
# Effects every function in this notebook
maximise = False

# Target Function

In [ ]:
sf = 1 if maximise else -1
f = lambda x: sf * (np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1))
xs = np.linspace(-2, 10, 1000)
ys = f(xs)
best_y = np.max(ys) if maximise else np.min(ys)

class TestEvaluator(op.Evaluator):
    def test_config(self, config):
        return f(config.x)
evaluator = TestEvaluator()

plt.figure(figsize=(16,6))
plt.plot(xs, ys)
plt.show()

# Helper Functions
These functions set up the optimiser to act like [this library](https://github.com/fmfn/BayesianOptimization/blob/master/examples/exploitation%20vs%20exploration.ipynb)

In [ ]:
def create_optimiser(ac_func, ac_args):
    '''
    create an optimiser which is configured to be as close those used by the
    Bayesian optimisation library this notebook is copying, with the acquisition
    function and arguments for it passed as parameters
    '''
    #ranges = {'x' : xs, 'y' : [1,2,3]}
    ranges = {'x' : xs}
    strat = op.AcquisitionStrategy(
        pre_phase_steps = 2,
        acquisition_function = (ac_func, ac_args),
        parallel_strategy = 'KB'
    )
    gp_params = {'alpha': 1e-5, 'kernel':1.0*gp.kernels.Matern(nu=2.5), 'n_restarts_optimizer':2}
    # these settings are very over-kill, the other library uses 250 restarts which is far too many
    maximiser_args = {'num_random' : 1e5, 'num_grad_restarts' : 10, 'take_best_random' : 3}
    
    return op.BayesianOptimisationOptimiser(
        ranges, maximise_cost=maximise,
        acquisition_strategy=strat,
        gp_params=gp_params,
        maximisation_args=maximiser_args,
        close_tolerance=1e-15)

def step_slider(optimiser):
    def plot(n, step):
        optimiser.plot_step_1D('x', n, true_cost=f);
    op.gui.step_log_slider(optimiser, plot, pre_compute=False)

def run_optimiser(ac_func, ac_args):
    ''' create an optimiser, run it and display the results '''
    bo = create_optimiser(ac_func, ac_args)
    op.gui.optimiser_progress_bar(bo, close_when_complete=True)
    bo.run_sequential(evaluator, max_jobs=25)
    step_slider(bo)
    return bo

# Upper Confidence Bound: Prefer Exploitation
$\kappa=1.0$

In [ ]:
bo = run_optimiser('UCB', {'kappa' : 1})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Upper Confidence Bound: Prefer Exploration
$\kappa=10.0$

In [ ]:
bo = run_optimiser('UCB', {'kappa' : 10})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Expected Value (Not Useful) (Upper Confidence Bound - Pure Exploitation)
$$\kappa=0$$
(only the mean/expected value $\mu$ is taken into account, the uncertainty ($\sigma$) is ignored)

In [ ]:
bo = run_optimiser('UCB', {'kappa' : 0})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Upper Confidence Bound - Pure Exploration (Not Useful)
$$\kappa=\infty$$ (only the uncertainty ($\sigma$) is taken into account, $\mu$ is ignored)

In [ ]:
bo = run_optimiser('UCB', {'kappa' : float('inf')})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Expected Improvement: Prefer Exploitation
$\xi=0.0001$

In [ ]:
bo = run_optimiser('EI', {'xi' : 1e-4})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Expected Improvement: Prefer Exploration
$\xi=0.1$

In [ ]:
bo = run_optimiser('EI', {'xi' : 0.1})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Probability of Improvement: Prefer Exploitation
$\xi=0.0001$

In [ ]:
bo = run_optimiser('PI', {'xi' : 1e-4})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Probability of Improvement: Prefer Exploration
$\xi=0.1$

In [ ]:
bo = run_optimiser('PI', {'xi' : 0.1})

In [ ]:
bo.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)